# Modelo KNN (K-nearest neighbors)

In [ ]:
# Import das dependências 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df = pd.read_csv("tabela_modelo.csv")

In [ ]:
# Remove a primeira coluna que está vazia
df.drop(df.columns[0], axis=1, inplace=True)
df

In [ ]:
df.head()

In [ ]:
# Conversão dos datetime para numeric

df['ID1 temp med'] = pd.to_numeric(df['ID1 temp med'], errors='coerce')
df['ID2 temp med'] = pd.to_numeric(df['ID2 temp med'], errors='coerce')
df['ID718 temp med'] = pd.to_numeric(df['ID718 temp med'], errors='coerce')

In [ ]:
df.info()

In [ ]:
# Normalização de colunas numéricas
columns_to_normalize = ['ID1 S OK', 'ID2 S OK', 'ID718 S OK', 'ID1 S NOK', 'ID2 S NOK', 'ID718 S NOK']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [ ]:
# Criar variáveis alvo e preditoras
df['TOTAL OK'] = df['ID1 S OK'] + df['ID2 S OK'] + df['ID718 S OK']
df['TOTAL NOK'] = df['ID1 S NOK'] + df['ID2 S NOK'] + df['ID718 S NOK']

# Definir a coluna 'RESULTADO' onde 1 = "OK" e 0 = "NOK" se houver ao menos um NOK
df['RESULTADO'] = (df['TOTAL NOK'] == 0).astype(int)

# Preparar os dados para a predição da falha
# Criar a coluna 'FALHA_OPER' indicando onde a falha pode ocorrer
df['FALHA_OPER'] = df[['ID1 S NOK', 'ID2 S NOK', 'ID718 S NOK']].idxmax(axis=1)
df['FALHA_OPER'] = df['FALHA_OPER'].map({'ID1 S NOK': 'ID1', 'ID2 S NOK': 'ID2', 'ID718 S NOK': 'ID718'})

# Se não houver nenhuma falha, definir 'FALHA_OPER' como 'Nenhuma'
df.loc[df['TOTAL NOK'] == 0, 'FALHA_OPER'] = 'Nenhuma'

# Features (X) e Target (y) para a predição de falha
X = df[['ID1 QT', 'ID1 temp med',
        'ID2 QT', 'ID2 temp med',
        'ID718 QT', 'ID718 temp med']]
y = df['FALHA_OPER']

# Transformar a variável alvo (y) em numérica para o modelo de classificação
y_encoded = y.map({'ID1': 0, 'ID2': 1, 'ID718': 2, 'Nenhuma': 3})

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_encoded, test_size=0.3, random_state=42)

# Treinar o modelo KNN
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

# Prever os resultados no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar a performance do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['ID1', 'ID2', 'ID718', 'Nenhuma']))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Adicionar a coluna 'FALHA_OPER' no DataFrame original com as previsões
df['FALHA_OPER'] = model.predict(imputer.transform(X))
df['FALHA_OPER'] = df['FALHA_OPER'].map({0: 'ID1', 1: 'ID2', 2: 'ID718', 3: 'Nenhuma'})

print("\nDataFrame com previsões:")
df

In [ ]:
# Criar a matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2, 3])

# Criar a visualização
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ID1', 'ID2', 'ID718', 'Nenhuma'])

# Plotar a matriz de confusão
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, cmap=plt.cm.Blues, colorbar=True)

# Adicionar títulos e labels
plt.title('Matriz de Confusão do Modelo KNN')
plt.show()